# Calculate Stats on Tuning Distributions

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthoc
from statsmodels.stats.multitest import multipletests
from hmmlearn import hmm

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_plotting as fp

In [2]:
def h5_to_dict(filepath):
    with pd.HDFStore(filepath, 'r') as store:
        keys = store.keys()
        data = {key[1:]: store[key] for key in keys}
    return data


def kruskal_w_dunn(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.kruskal(*data, nan_policy=nan_policy)
    dunn_test = posthoc.posthoc_dunn([*data], p_adjust='holm')

    # Return just the corrected pvals instead of a whole matrix
    if dunn_test.shape[-1] > 2:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])] + [dunn_test.iloc[1, -1]])
    else:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])])
    return p_val, p_val_corrected


def mannwhitneyu_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.mannwhitneyu(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def wilcoxon_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.wilcoxon(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def sample_df(data1, data2):
    df1 = data1.dropna()
    df2 = data2.dropna() 

    if len(df1) > len(df2):
        df1 = df1.sample(len(df2), replace=True)
    elif len(df2) > len(df1):
        df2 = df2.sample(len(df1), replace=True)
    else:
        pass

    return df1, df2


def bootstrap_tests(datasets, key, tuning_kind, method, n_iters=10000, alpha=0.05, bootstrap_all=False):
    data = [ds[key][tuning_kind].dropna() for ds in datasets]

    # First run the test on the raw data
    if 'wilcoxon' in method.__name__:
        raw_test = method(sample_df(*data), alpha=alpha)
    elif 'mannwhitneyu' in method.__name__:
        raw_test = method(data, alpha=alpha)
    else:
        raw_test = method(data, alpha=alpha)

    # Now run the bootstrapping
    p_vals = np.zeros(n_iters)
    p_vals_corrected = np.zeros((n_iters, len(data)))

    sample_size = len(max(data, key=len))

    # Run bootstrap
    for i in range(n_iters):

        if bootstrap_all:
            sample_data = [d.sample(sample_size, replace=True) for d in data]
            # sample_data1  = data2.sample(sample_size, replace=True)
        else:
            sample_data = [data[0]] + [d.sample(sample_size, replace=True) for d in data[1:]]
            
        p_val, p_val_corrected = method(sample_data, alpha=alpha)
        p_vals[i] = p_val
        p_vals_corrected[i, :] = p_val_corrected

    # 95% confidence interval of pvals
    ci = stats.t.interval(0.95, len(p_vals) - 1, loc=np.nanmean(p_vals), scale=stats.sem(p_vals, nan_policy='omit'))

    if method == kruskal_w_dunn:
        # Since this is for multiple groups, need to run this column-wise

        # Corrected 95% CIs
        ci_corrected = np.zeros((len(data), 2))
        for i in range(p_vals_corrected.shape[-1]):
            p_val_corr_col = p_vals_corrected[:, i]
            ci_col = stats.t.interval(0.95, len(p_val_corr_col) - 1, loc=np.nanmean(p_val_corr_col), scale=stats.sem(p_val_corr_col, nan_policy='omit'))
            ci_corrected[i, :] = ci_col
    else:
        ci_corrected = stats.t.interval(0.95, len(p_vals_corrected) - 1, loc=np.nanmean(p_vals_corrected), scale=stats.sem(p_vals_corrected, nan_policy='omit'))

    return raw_test, (p_vals, ci), (p_vals_corrected, ci_corrected)


def plot_bootstrap_distributions(data):
    # Plot the bootstrap distributions
    num_ds = len(data)
    cols = 3
    rows = num_ds//cols + 1
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    for i, (ds, ax) in enumerate(zip(data, axes.ravel())):

        raw_test = ds[0]
        bootstrap_tests = ds[1]
        corrected_bootstrap_tests = ds[2]

        ax.hist(bootstrap_tests[0], bins=20)
        ax.axvline(np.percentile(bootstrap_tests[0], 95), 0, 50, color='r')
        ax.axvline(bootstrap_tests[1][0], 0, 50, color='g')
        ax.axvline(bootstrap_tests[1][1], 0, 50, color='g')
        ax.axvline(raw_test[0], 0, 100, color='k')


    return fig, axes

In [3]:
basedir = r"Z:\Prey_capture\WF_Figures"
still_or_all = 'full'
main_exp_path = os.path.join(basedir, still_or_all, r"multi_normal_ALL\stats.hdf5")
control_light_path  = os.path.join(basedir, still_or_all, r"control_normal_ALL\stats.hdf5")
control_dark_path = os.path.join(basedir, still_or_all, r"control_dark_ALL\stats.hdf5")
repeat_fixed_path = os.path.join(basedir, still_or_all, r"repeat_normal_VWheelWF\stats.hdf5")
repeat_free_path = os.path.join(basedir, still_or_all, r"repeat_normal_VTuningWF\stats.hdf5")
output_path = os.path.join(basedir, 'stats', f"{still_or_all}_tuning_stats.h5")

In [4]:
main_exp_data = h5_to_dict(main_exp_path)
control_light_data = h5_to_dict(control_light_path)
control_dark_data = h5_to_dict(control_dark_path)
repeat_fixed_data = h5_to_dict(repeat_fixed_path)
repeat_free_data = h5_to_dict(repeat_free_path)

In [5]:
method = kruskal_w_dunn
bootstrap_iters = 1000

# Kinematic Tuning

In [6]:
def kinem_stats(data, ds_name, alpha=0.05):
    cols = data[0][ds_name].columns.to_list()

    stats = {}
    for col in cols:
        stats[col] = bootstrap_tests(data, ds_name, col,   
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)

    return stats

In [7]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_kinem_tuning_free']

#     for var in a.index.to_list():
#         print(var)
#         print(a.loc[var, 'control_dark'][-1])

In [8]:
# Fixed data
fixed_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_fixed')
kinem_tuning_fixed = pd.DataFrame(fixed_kinem).T
kinem_tuning_fixed.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


# Free data
free_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_free')
kinem_tuning_free = pd.DataFrame(free_kinem).T
kinem_tuning_free.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_free'] = kinem_tuning_free


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4136049986.py:8: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['main', 'control_light', 'control_dark'], dtype='object')]

  store[f'kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4136049986.py:18: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['main', 'control_light', 'control_dark'], dtype='object')]

  store[f'kruskal_kinem_tuning_free'] = kinem_tuning_free


# Visual Tuning

In [9]:
vis_classes = ['', 'mod_', 'not_']

## Frac Visual Tuning - Fixed to Free

### Kruskal-Wallis

In [10]:
for vis_class in vis_classes:
    # Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

    # Visual
    exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (0.6368039160463146, array([0.63680392]))
Fixed vs Free Orientation: (5.562268506118308e-13, array([5.56226851e-13]))
Fixed vs Free Direction: (0.039924934730025854, array([0.03992493]))
Fixed vs Free Visual: (2.3224044590229072e-07, array([2.32240446e-07]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types

Fixed vs Free Orientation: (0.03587265361570818, array([0.03587265]))
Fixed vs Free Direction: (0.07335503543167632, array([0.07335504]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types

Fixed vs Free Visual: (1.6615940515672703e-11, array([1.66159405e-11]))
Fixed vs Free Orientation: (1.0729285688929223e-09, array([1.07292857e-09]))
Fixed vs Free Direction: (1.6170257352397262e-06, array([1.61702574e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\93519829.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types

### Wilcoxon Rank Sum

In [11]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.5535240471550497, array([0.55352405]))
Fixed vs Free Orientation: (2.1622325721292424e-17, array([2.16223257e-17]))
Fixed vs Free Direction: (0.0035842744078428378, array([0.00358427]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

Fixed vs Free Visual: (4.926295351994665e-09, array([4.92629535e-09]))
Fixed vs Free Orientation: (0.024496672188132375, array([0.02449667]))
Fixed vs Free Direction: (0.06942877646669475, array([0.06942878]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

Fixed vs Free Visual: (4.549549164014347e-20, array([4.54954916e-20]))
Fixed vs Free Orientation: (9.044640840395366e-13, array([9.04464084e-13]))
Fixed vs Free Direction: (9.382871325416073e-11, array([9.38287133e-11]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\4259356661.py:25: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-t

### Mann-Whitney U

In [12]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

Fixed vs Free Visual: (0.6369949704544589, array([0.63699497]))
Fixed vs Free Orientation: (5.573587631945802e-13, array([5.57358763e-13]))
Fixed vs Free Direction: (0.03995130385570297, array([0.0399513]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

Fixed vs Free Visual: (2.3257271126334626e-07, array([2.32572711e-07]))
Fixed vs Free Orientation: (0.03589671204574371, array([0.03589671]))
Fixed vs Free Direction: (0.07339811688855885, array([0.07339812]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:20: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1293581978.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directl

## Frac Visual Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [13]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI', frac_fixed_vis[1][1])
    print('Fixed vs Controls Visual 95% CI - Corrected:', frac_fixed_vis[-1][1])
    print('Fixed vs Controls Visual Raw:', frac_fixed_vis[0])

    # Orientation
    frac_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Ori 95% CI', frac_fixed_ori[1][1])
    print('Fixed vs Controls Ori 95% CI - Corrected:', frac_fixed_ori[-1][1])
    print('Fixed vs Controls Ori Raw:', frac_fixed_ori[0])

    # Direction
    frac_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Dir 95% CI', frac_fixed_dir[1][1])
    print('Fixed vs Controls Dir 95% CI - Corrected:', frac_fixed_dir[-1][1])
    print('Fixed vs Controls Dir Raw:', frac_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_dir'] = pd.DataFrame(list(frac_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI (-4.219527126916455e-25, 1.2989254494384436e-24)
Fixed vs Controls Visual 95% CI - Corrected: [[-1.51802415e-16  4.67306565e-16]
 [-2.26808209e-23  1.32859905e-22]
 [ 4.19816074e-01  4.55552292e-01]]
Fixed vs Controls Visual Raw: (8.366268871528825e-15, array([1.57344464e-07, 9.83506935e-11, 6.07587104e-01]))
Fixed vs Controls Ori 95% CI (-1.0636906967236118e-68, 7.913360329623997e-68)
Fixed vs Controls Ori 95% CI - Corrected: [[-6.65599890e-54  4.08754680e-53]
 [-1.72917781e-50  1.64163652e-49]
 [ 4.91761390e-01  5.21395961e-01]]
Fixed vs Controls Ori Raw: (1.4468502067743607e-19, array([3.18220818e-11, 1.40078048e-12, 8.89164817e-01]))
Fixed vs Controls Dir 95% CI (-4.3012065535100076e-33, 1.421860057360119e-32)
Fixed vs Controls Dir 95% CI - Corrected: [[-8.73570228e-31  3.99797150e-30]
 [-2.25962428e-21  2.87911910e-20]
 [ 9.91548439e-02  1.17341937e-01]]
Fixed vs Controls Dir Raw: (2.9913270159495296e-13, array([2.46097057e-08, 3.44884283e-08, 6.066

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:33: PerformanceWarning: 
your performance may suff

Fixed vs Controls Visual 95% CI (-1.1569772824988846e-28, 3.6814519101355353e-28)
Fixed vs Controls Visual 95% CI - Corrected: [[-5.65670454e-20  1.82091678e-19]
 [-3.99988624e-24  2.79616239e-23]
 [ 4.38124489e-01  4.73993771e-01]]
Fixed vs Controls Visual Raw: (8.366268871528825e-15, array([1.57344464e-07, 9.83506935e-11, 6.07587104e-01]))
Fixed vs Controls Ori 95% CI (-1.8342444679239147e-62, 1.0954879555280172e-61)
Fixed vs Controls Ori 95% CI - Corrected: [[-1.06486468e-50  3.32862007e-50]
 [-5.43411620e-44  1.68488931e-43]
 [ 4.89940783e-01  5.19487394e-01]]
Fixed vs Controls Ori Raw: (1.4468502067743607e-19, array([3.18220818e-11, 1.40078048e-12, 8.89164817e-01]))
Fixed vs Controls Dir 95% CI (-1.6070419341079596e-32, 6.629091285711173e-32)
Fixed vs Controls Dir 95% CI - Corrected: [[-7.69260996e-29  2.46972727e-28]
 [-1.50301225e-21  1.47636384e-20]
 [ 9.67120983e-02  1.14776608e-01]]
Fixed vs Controls Dir Raw: (2.9913270159495296e-13, array([2.46097057e-08, 3.44884283e-08, 6.0

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:33: PerformanceWarning: 
your performance may suff

Fixed vs Controls Visual 95% CI (-2.0185599810085138e-27, 6.214787138848481e-27)
Fixed vs Controls Visual 95% CI - Corrected: [[-5.45997448e-22  3.56180864e-20]
 [-5.66155457e-23  1.76048362e-22]
 [ 4.42936029e-01  4.79224670e-01]]
Fixed vs Controls Visual Raw: (8.366268871528825e-15, array([1.57344464e-07, 9.83506935e-11, 6.07587104e-01]))
Fixed vs Controls Ori 95% CI (1.850931064218834e-69, 1.5091532065409043e-67)
Fixed vs Controls Ori 95% CI - Corrected: [[-1.43546385e-54  5.45614300e-54]
 [-2.42735267e-48  1.13155035e-47]
 [ 4.90091043e-01  5.18645202e-01]]
Fixed vs Controls Ori Raw: (1.4468502067743607e-19, array([3.18220818e-11, 1.40078048e-12, 8.89164817e-01]))
Fixed vs Controls Dir 95% CI (-2.8793841375639956e-35, 1.0562255480113369e-34)
Fixed vs Controls Dir 95% CI - Corrected: [[-6.56937887e-30  2.05018762e-29]
 [-3.84329221e-20  1.25527328e-19]
 [ 8.88894555e-02  1.04931858e-01]]
Fixed vs Controls Dir Raw: (2.9913270159495296e-13, array([2.46097057e-08, 3.44884283e-08, 6.066

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:31: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1589276187.py:33: PerformanceWarning: 
your performance may suff

In [14]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store[f'{vis_class}vis/kruskal_frac_fixed_dir']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

In [15]:
# frac_fixed_fig = plot_bootstrap_distributions([frac_fixed_gen, 
#                                                frac_fixed_vis, frac_fixed_vis_no_gen, 
#                                                frac_fixed_ori, frac_fixed_ori_no_gen,
#                                                frac_fixed_dir, frac_fixed_dir_no_gen])
# plt.show()

### Free Kruskal Wallis

In [16]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI', frac_free_vis[1][1])
    print('Free vs Controls Visual 95% CI - Corrected:', frac_free_vis[-1][1])
    print('Free vs Controls Visual Raw:', frac_free_vis[0])

    # Orientation
    frac_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Ori 95% CI', frac_free_ori[1][1])
    print('Free vs Controls Ori 95% CI - Corrected:', frac_free_ori[-1][1])
    print('Free vs Controls Ori Raw:', frac_free_ori[0])

    # Direction
    frac_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Dir 95% CI', frac_free_dir[1][1])
    print('Free vs Controls Dir 95% CI - Corrected:', frac_free_dir[-1][1])
    print('Free vs Controls Dir Raw:', frac_free_dir[0])


    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_dir'] = pd.DataFrame(list(frac_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI (-2.293552065648655e-24, 3.202042221537334e-23)
Free vs Controls Visual 95% CI - Corrected: [[-3.99624905e-17  9.32551390e-16]
 [-5.32818722e-21  7.36856654e-20]
 [ 3.35292329e-01  3.70159357e-01]]
Free vs Controls Visual Raw: (2.256976482633488e-12, array([7.64863616e-07, 1.22705793e-08, 8.38302451e-01]))
Free vs Controls Ori 95% CI (-2.11927911358321e-18, 3.893333413705232e-17)
Free vs Controls Ori 95% CI - Corrected: [[ 1.53993090e-08  2.27089377e-07]
 [-8.40208193e-18  4.61750709e-17]
 [ 8.09009366e-03  1.23092298e-02]]
Free vs Controls Ori Raw: (1.215172622536174e-07, array([1.63979021e-03, 2.42977059e-06, 3.70787550e-01]))
Free vs Controls Dir 95% CI (-1.4208155342293747e-19, 4.462757231138728e-19)
Free vs Controls Dir 95% CI - Corrected: [[ 2.30181264e-06  1.30035432e-05]
 [-5.54512750e-20  1.74336034e-19]
 [ 1.06225633e-05  3.69369105e-05]]
Free vs Controls Dir Raw: (6.262589083172232e-09, array([4.42860952e-03, 4.12671462e-08, 9.76719490e-02]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

Free vs Controls Visual 95% CI (-2.2903172318774863e-22, 8.257117647610215e-22)
Free vs Controls Visual 95% CI - Corrected: [[-4.08220293e-16  3.92954557e-15]
 [-6.64653371e-19  2.70270429e-18]
 [ 3.46604739e-01  3.82142525e-01]]
Free vs Controls Visual Raw: (2.256976482633488e-12, array([7.64863616e-07, 1.22705793e-08, 8.38302451e-01]))
Free vs Controls Ori 95% CI (-2.4016967534712182e-17, 1.714797117192958e-16)
Free vs Controls Ori 95% CI - Corrected: [[-5.57796259e-08  5.89275023e-07]
 [-2.30529380e-17  1.69191412e-16]
 [ 7.33341625e-03  1.05178693e-02]]
Free vs Controls Ori Raw: (1.215172622536174e-07, array([1.63979021e-03, 2.42977059e-06, 3.70787550e-01]))
Free vs Controls Dir 95% CI (-2.1571559548748927e-23, 5.383599068347666e-21)
Free vs Controls Dir 95% CI - Corrected: [[ 2.04386016e-06  1.74271193e-05]
 [-4.72957162e-23  1.68295766e-21]
 [ 7.38681214e-06  1.96197522e-05]]
Free vs Controls Dir Raw: (6.262589083172232e-09, array([4.42860952e-03, 4.12671462e-08, 9.76719490e-02])

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

Free vs Controls Visual 95% CI (-1.8149404102245638e-19, 5.588185986573532e-19)
Free vs Controls Visual 95% CI - Corrected: [[-1.01077514e-14  3.45453837e-14]
 [-2.18609153e-17  6.92461335e-17]
 [ 3.37533010e-01  3.72058173e-01]]
Free vs Controls Visual Raw: (2.256976482633488e-12, array([7.64863616e-07, 1.22705793e-08, 8.38302451e-01]))
Free vs Controls Ori 95% CI (1.5099301866604638e-18, 6.879028375583995e-17)
Free vs Controls Ori 95% CI - Corrected: [[-1.74980158e-07  1.07017463e-06]
 [-1.21416337e-18  6.52293351e-17]
 [ 6.71690798e-03  1.05030008e-02]]
Free vs Controls Ori Raw: (1.215172622536174e-07, array([1.63979021e-03, 2.42977059e-06, 3.70787550e-01]))
Free vs Controls Dir 95% CI (-4.294102840342218e-19, 1.3913044790794687e-18)
Free vs Controls Dir 95% CI - Corrected: [[-2.51336381e-05  8.97180641e-05]
 [-1.26110234e-19  4.34334560e-19]
 [ 7.63207363e-06  2.42823288e-05]]
Free vs Controls Dir Raw: (6.262589083172232e-09, array([4.42860952e-03, 4.12671462e-08, 9.76719490e-02]))

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:32: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\361439579.py:34: PerformanceWarning: 
your performance may suffer as P

In [17]:
# frac_free_fig = plot_bootstrap_distributions([frac_free_gen, 
#                                               frac_free_vis, frac_free_vis_no_gen, 
#                                               frac_free_ori, frac_free_ori_no_gen,
#                                               frac_free_dir, frac_free_dir_no_gen])
# plt.show()

## Count Vis Tuning - Fixed vs Free

### Kruskal-Wallis

In [18]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    count_exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1890522773.py:24: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

### Wilcoxon Rank Sum

In [19]:
# Do a Wilcox Rank-Sum test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))
Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))
Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\553109606.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it 

### Mann-Whitney U

In [20]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))
Fixed vs Free Visual:

C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

 (1.6646842229142308e-11, array([1.66468422e-11]))
Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))
Fixed vs Free Visual: (1.6646842229142308e-11, array([1.66468422e-11]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

Fixed vs Free Orientation: (1.0751165968978945e-09, array([1.0751166e-09]))
Fixed vs Free Direction: (1.6193600592145545e-06, array([1.61936006e-06]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:22: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Int64Index([0], dtype='int64')]

  store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\920968691.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types 

## Count Vis Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [21]:
# Do a Kruskal-Wallis test with Dunn posthoc

for vis_class in vis_classes:

    # Visual
    count_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI:', count_fixed_vis[1][1])
    print('Fixed vs Controls Visual Raw:', count_fixed_vis[0])

    # Orientation
    count_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Orientation 95% CI:', count_fixed_ori[1][1])
    print('Fixed vs Controls Orientation Raw:', count_fixed_ori[0])

    # Direction
    count_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Direction 95% CI:', count_fixed_dir[1][1])
    print('Fixed vs Controls Direction Raw:', count_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_dir'] = pd.DataFrame(list(count_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Visual 95% CI: (-2.567150223488718e-07, 2.5353063432427752e-06)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-1.5854064097329855e-21, 6.423371191100037e-21)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-3.651593254821519e-11, 4.784199111506427e-10)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:32: PerformanceWarning: 
your performance may 

Fixed vs Controls Visual 95% CI: (1.5281081180699622e-08, 2.9232665895948906e-06)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-2.1990657834116196e-21, 1.5024818800168422e-20)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (4.978197228724627e-12, 6.882987011076393e-11)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:32: PerformanceWarning: 
your performance may 

Fixed vs Controls Visual 95% CI: (1.7309893533918707e-07, 2.1619940348969113e-06)
Fixed vs Controls Visual Raw: (0.000268196058097534, array([0.00634131, 0.00634131, 0.78859931]))
Fixed vs Controls Orientation 95% CI: (-5.498434728016464e-22, 1.8149164428726968e-21)
Fixed vs Controls Orientation Raw: (5.443136292223412e-08, array([5.21956459e-05, 2.30637753e-05, 9.07642115e-01]))
Fixed vs Controls Direction 95% CI: (-5.086446227485963e-11, 2.385089461903857e-10)
Fixed vs Controls Direction Raw: (1.6802916286478037e-05, array([0.00106744, 0.00106744, 0.73690399]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:28: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:30: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\1713287464.py:32: PerformanceWarning: 
your performance may 

### Free Kruskal Wallis

In [22]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:

    # Visual
    count_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI:', count_free_vis[1][1])
    print('Free vs Controls Visual Raw:', count_free_vis[0])

    # Orientation
    count_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Orientation 95% CI:', count_free_ori[1][1])
    print('Free vs Controls Orientation Raw:', count_free_ori[0])

    # Direction
    count_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Direction 95% CI:', count_free_dir[1][1])
    print('Free vs Controls Direction Raw:', count_free_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_dir'] = pd.DataFrame(list(count_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Visual 95% CI: (5.085348331800605e-07, 3.2991142693782306e-06)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (2.4379307677528055e-07, 2.562021680041131e-06)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (-1.5220674557641285e-08, 1.0733916024516493e-07)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:31: PerformanceWarning: 
your performance may suff

Free vs Controls Visual 95% CI: (-4.506344429273759e-07, 6.354160976440384e-06)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (3.22141134346072e-07, 3.1570008041507363e-06)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (1.3331842653378983e-09, 1.0521336257520351e-08)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:31: PerformanceWarning: 
your performance may suff

Free vs Controls Visual 95% CI: (6.227205626339049e-08, 5.851732944823053e-06)
Free vs Controls Visual Raw: (0.0003101948186057097, array([0.01768822, 0.00235345, 0.73407518]))
Free vs Controls Orientation 95% CI: (-2.701507645025714e-07, 2.4278217762924573e-06)
Free vs Controls Orientation Raw: (0.001265652736278355, array([0.06810468, 0.00383673, 0.5248231 ]))
Free vs Controls Direction 95% CI: (-1.6912994107270064e-08, 1.0380331964495544e-07)
Free vs Controls Direction Raw: (0.0002634136057045376, array([0.08240132, 0.00055088, 0.28396112]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:27: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:29: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\2513833730.py:31: PerformanceWarning: 
your performance may suff

# Running Modulated

## Fixed

In [23]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_fixed_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_fixed', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Run Mod. 95% CI', frac_fixed_run_mod[1][1])
print('Fixed vs Controls Run Mod. 95% CI - Corrected:', frac_fixed_run_mod[-1][1])
print('Fixed vs Controls Run Mod. Raw:', frac_fixed_run_mod[0])

# Running and Visually Modulated
frac_fixed_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_fixed', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Vis. + Run Mod. 95% CI', frac_fixed_vis_run_mod[1][1])
print('Fixed vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_fixed_vis_run_mod[-1][1])
print('Fixed vs Controls Vis. + Run Mod. Raw:', frac_fixed_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_fixed_run_mod'] = pd.DataFrame(list(frac_fixed_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_fixed_vis_run_mod'] = pd.DataFrame(list(frac_fixed_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Fixed vs Controls Run Mod. 95% CI (0.3557069668175729, 0.3928559554587849)
Fixed vs Controls Run Mod. 95% CI - Corrected: [[0.61118157 0.65420881]
 [0.65731332 0.69693776]
 [0.56272833 0.60856571]]
Fixed vs Controls Run Mod. Raw: (0.9442514760228817, array([1., 1., 1.]))
Fixed vs Controls Vis. + Run Mod. 95% CI (0.462767944784378, 0.4997216212971214)
Fixed vs Controls Vis. + Run Mod. 95% CI - Corrected: [[0.7095718  0.74890941]
 [0.72747871 0.76502486]
 [0.71289749 0.75162364]]
Fixed vs Controls Vis. + Run Mod. Raw: (0.9930991992130431, array([1., 1., 1.]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\193980662.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_fixed_run_mod'] = pd.DataFrame(list(frac_fixed_run_mod), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\193980662.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_fixed_vis_run_mod'] = pd.DataFrame(list(frac_fixed_vis_run_mod), columns=['pval', 'ci'],


In [24]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_frac_free_run_mod']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

## Free

In [25]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_free_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_free', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Run Mod. 95% CI', frac_free_run_mod[1][1])
print('Free vs Controls Run Mod. 95% CI - Corrected:', frac_free_run_mod[-1][1])
print('Free vs Controls Run Mod. Raw:', frac_free_run_mod[0])

# Running and Visually Modulated
frac_free_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_free', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Vis. + Run Mod. 95% CI', frac_free_vis_run_mod[1][1])
print('Free vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_free_vis_run_mod[-1][1])
print('Free vs Controls Vis. + Run Mod. Raw:', frac_free_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_free_run_mod'] = pd.DataFrame(list(frac_free_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_free_vis_run_mod'] = pd.DataFrame(list(frac_free_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

Free vs Controls Run Mod. 95% CI (0.015448591707197656, 0.023272150925609743)
Free vs Controls Run Mod. 95% CI - Corrected: [[0.39710937 0.43439417]
 [0.0309366  0.0443857 ]
 [0.08043503 0.10006718]]
Free vs Controls Run Mod. Raw: (0.20904135793189865, array([0.70912232, 0.23373511, 0.66223755]))
Free vs Controls Vis. + Run Mod. 95% CI (0.257088319504758, 0.29112995740918973)
Free vs Controls Vis. + Run Mod. 95% CI - Corrected: [[0.55325688 0.59575901]
 [0.40167317 0.44856603]
 [0.55335567 0.5940595 ]]
Free vs Controls Vis. + Run Mod. Raw: (0.6308430619522412, array([1., 1., 1.]))


C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\602213168.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_free_run_mod'] = pd.DataFrame(list(frac_free_run_mod), columns=['pval', 'ci'],
C:\Users\mmccann\AppData\Local\Temp\ipykernel_26392\602213168.py:23: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests'], dtype='object')]

  store[f'kruskal_frac_free_vis_run_mod'] = pd.DataFrame(list(frac_free_vis_run_mod), columns=['pval', 'ci'],
